# Starbucks Capstone Challenge

### Introduction

This data set contains simulated data that mimics customer behavior on the Starbucks rewards mobile app. Once every few days, Starbucks sends out an offer to users of the mobile app. An offer can be merely an advertisement for a drink or an actual offer such as a discount or BOGO (buy one get one free). Some users might not receive any offer during certain weeks. 

Not all users receive the same offer, and that is the challenge to solve with this data set.

Your task is to combine transaction, demographic and offer data to determine which demographic groups respond best to which offer type. This data set is a simplified version of the real Starbucks app because the underlying simulator only has one product whereas Starbucks actually sells dozens of products.

Every offer has a validity period before the offer expires. As an example, a BOGO offer might be valid for only 5 days. You'll see in the data set that informational offers have a validity period even though these ads are merely providing information about a product; for example, if an informational offer has 7 days of validity, you can assume the customer is feeling the influence of the offer for 7 days after receiving the advertisement.

You'll be given transactional data showing user purchases made on the app including the timestamp of purchase and the amount of money spent on a purchase. This transactional data also has a record for each offer that a user receives as well as a record for when a user actually views the offer. There are also records for when a user completes an offer. 

Keep in mind as well that someone using the app might make a purchase through the app without having received an offer or seen an offer.

### Example

To give an example, a user could receive a discount offer buy 10 dollars get 2 off on Monday. The offer is valid for 10 days from receipt. If the customer accumulates at least 10 dollars in purchases during the validity period, the customer completes the offer.

However, there are a few things to watch out for in this data set. Customers do not opt into the offers that they receive; in other words, a user can receive an offer, never actually view the offer, and still complete the offer. For example, a user might receive the "buy 10 dollars get 2 dollars off offer", but the user never opens the offer during the 10 day validity period. The customer spends 15 dollars during those ten days. There will be an offer completion record in the data set; however, the customer was not influenced by the offer because the customer never viewed the offer.

### Cleaning

This makes data cleaning especially important and tricky.

You'll also want to take into account that some demographic groups will make purchases even if they don't receive an offer. From a business perspective, if a customer is going to make a 10 dollar purchase without an offer anyway, you wouldn't want to send a buy 10 dollars get 2 dollars off offer. You'll want to try to assess what a certain demographic group will buy when not receiving any offers.

### Final Advice

Because this is a capstone project, you are free to analyze the data any way you see fit. For example, you could build a machine learning model that predicts how much someone will spend based on demographics and offer type. Or you could build a model that predicts whether or not someone will respond to an offer. Or, you don't need to build a machine learning model at all. You could develop a set of heuristics that determine what offer you should send to each customer (i.e., 75 percent of women customers who were 35 years old responded to offer A vs 40 percent from the same demographic to offer B, so send offer A).

# Data Sets

The data is contained in three files:

* portfolio.json - containing offer ids and meta data about each offer (duration, type, etc.)
* profile.json - demographic data for each customer
* transcript.json - records for transactions, offers received, offers viewed, and offers completed

Here is the schema and explanation of each variable in the files:

**portfolio.json**
* id (string) - offer id
* offer_type (string) - type of offer ie BOGO, discount, informational
* difficulty (int) - minimum required spend to complete an offer
* reward (int) - reward given for completing an offer
* duration (int) - time for offer to be open, in days
* channels (list of strings)

**profile.json**
* age (int) - age of the customer 
* became_member_on (int) - date when customer created an app account
* gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
* id (str) - customer id
* income (float) - customer's income

**transcript.json**
* event (str) - record description (ie transaction, offer received, offer viewed, etc.)
* person (str) - customer id
* time (int) - time in hours since start of test. The data begins at time t=0
* value - (dict of strings) - either an offer id or transaction amount depending on the record

**Note:** If you are using the workspace, you will need to go to the terminal and run the command `conda update pandas` before reading in the files. This is because the version of pandas in the workspace cannot read in the transcript.json file correctly, but the newest version of pandas can. You can access the termnal from the orange icon in the top left of this notebook.  

You can see how to access the terminal and how the install works using the two images below.  First you need to access the terminal:

<img src="pic1.png"/>

Then you will want to run the above command:

<img src="pic2.png"/>

Finally, when you enter back into the notebook (use the jupyter icon again), you should be able to run the below cell without any errors.

In [29]:
import pandas as pd
import numpy as np
import math
import json
import matplotlib.pyplot as plt
import seaborn as sns

# read in the json files
portfolio_raw = pd.read_json('data/portfolio.json', orient='records', lines=True)
profile_raw = pd.read_json('data/profile.json', orient='records', lines=True)
transcript_raw = pd.read_json('data/transcript.json', orient='records', lines=True)

## Data understanding



**portfolio.json**
* id (string) - offer id
* offer_type (string) - type of offer ie BOGO, discount, informational
* difficulty (int) - minimum required spend to complete an offer
* reward (int) - reward given for completing an offer
* duration (int) - time for offer to be open, in days
* channels (list of strings)


** -> Data is cleaned and ready to be used**

In [30]:
portfolio_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   reward      10 non-null     int64 
 1   channels    10 non-null     object
 2   difficulty  10 non-null     int64 
 3   duration    10 non-null     int64 
 4   offer_type  10 non-null     object
 5   id          10 non-null     object
dtypes: int64(3), object(3)
memory usage: 612.0+ bytes


In [31]:
portfolio_raw

,reward,channels,difficulty,duration,offer_type,id
0,10,"[email, mobile, social]",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd
1,10,"[web, email, mobile, social]",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0
2,0,"[web, email, mobile]",0,4,informational,3f207df678b143eea3cee63160fa8bed
3,5,"[web, email, mobile]",5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9
4,5,"[web, email]",20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7
5,3,"[web, email, mobile, social]",7,7,discount,2298d6c36e964ae4a3e7e9706d1fb8c2
6,2,"[web, email, mobile, social]",10,10,discount,fafdcd668e3743c1bb461111dcafc2a4
7,0,"[email, mobile, social]",0,3,informational,5a8bc65990b245e5a138643cd4eb9837
8,5,"[web, email, mobile, social]",5,5,bogo,f19421c1d4aa40978ebb69ca19b0e20d
9,2,"[web, email, mobile]",10,7,discount,2906b810c7d4411798c6938adc9daaa5


In [32]:
# portfolio_raw['channels'].explode().unique()

**profile.json**
* age (int) - age of the customer 
* became_member_on (int) - date when customer created an app account
* gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
* id (str) - customer id
* income (float) - customer's income

**-> the NoneType gender will be removed from profile dataset and storage in a new dataset**

In [33]:
profile_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17000 entries, 0 to 16999
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gender            14825 non-null  object 
 1   age               17000 non-null  int64  
 2   id                17000 non-null  object 
 3   became_member_on  17000 non-null  int64  
 4   income            14825 non-null  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 664.2+ KB


**transcript.json**
* event (str) - record description (ie transaction, offer received, offer viewed, etc.)
* person (str) - customer id
* time (int) - time in hours since start of test. The data begins at time t=0
* value - (dict of strings) - either an offer id or transaction amount depending on the record

In [34]:
transcript_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306534 entries, 0 to 306533
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   person  306534 non-null  object
 1   event   306534 non-null  object
 2   value   306534 non-null  object
 3   time    306534 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 9.4+ MB


In [35]:
transcript_raw['event'].value_counts()

event
transaction        138953
offer received      76277
offer viewed        57725
offer completed     33579
Name: count, dtype: int64

In [36]:
{[*x][0] for x in transcript_raw['value']}

{'amount', 'offer id', 'offer_id'}

### Data Cleaning and Formatting Plan

#### **Objective**
- Clean and standardize the dataset while handling null values appropriately.

---

#### **Profile Dataset**
1. Split the `profile` dataset into two subsets:
   - One containing rows where the `gender` is `none`.
   - Another containing rows with all other `gender` values.

2. Convert the `became_member_on` column to a standardized **datetime** format for consistency and easier analysis.

---

#### **Portfolio Dataset**
- Update column names to make them more descriptive and easier to read.

---

#### **Transcript Dataset**
- Modify the keys within the `value` dictionaries by replacing spaces (`' '`) with underscores (`'_'`) to ensure uniformity and avoid potential issues in further processing.

---

In [37]:
profile = profile_raw.copy(deep=True)

# Convert the 'became_member_on' column to a datetime format
profile['became_member_on'] = pd.to_datetime(profile['became_member_on'], format='%Y%m%d')

# Create a new column with only the year and month of the membership
profile['bec_memb_year_month'] = pd.to_datetime(profile['became_member_on'], format='%Y%m%d').dt.strftime('%Y-%m')

In [38]:
profile.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17000 entries, 0 to 16999
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   gender               14825 non-null  object        
 1   age                  17000 non-null  int64         
 2   id                   17000 non-null  object        
 3   became_member_on     17000 non-null  datetime64[ns]
 4   income               14825 non-null  float64       
 5   bec_memb_year_month  17000 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(3)
memory usage: 797.0+ KB


In [39]:
portfolio = portfolio_raw.copy()

port_id = {
    'ae264e3637204a6fb9bb56bc8210ddfd': 'ofr_A',
    '4d5c57ea9a6940dd891ad53e9dbe8da0': 'ofr_B',
    '3f207df678b143eea3cee63160fa8bed': 'ofr_C',
    '9b98b8c7a33c4b65b9aebfe6a799e6d9': 'ofr_D',
    '0b1e1539f2cc45b7b9fa7c272da2e1d7': 'ofr_E',
    '2298d6c36e964ae4a3e7e9706d1fb8c2': 'ofr_F',
    'fafdcd668e3743c1bb461111dcafc2a4': 'ofr_G',
    '5a8bc65990b245e5a138643cd4eb9837': 'ofr_H',
    'f19421c1d4aa40978ebb69ca19b0e20d': 'ofr_I',
    '2906b810c7d4411798c6938adc9daaa5': 'ofr_J'
}

portfolio['ofr_id_short'] = portfolio['id'].map(port_id)

portfolio.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   reward        10 non-null     int64 
 1   channels      10 non-null     object
 2   difficulty    10 non-null     int64 
 3   duration      10 non-null     int64 
 4   offer_type    10 non-null     object
 5   id            10 non-null     object
 6   ofr_id_short  10 non-null     object
dtypes: int64(3), object(4)
memory usage: 692.0+ bytes


# Data Preprocessing and Transformation Pipeline (ETL)
---
- Objective:

Creation of the `Analytical Table` dataset for machine learning applications.

- Strategy:
1. Loading raw data from the original tables.
2. Restructuring it using groupby/unstack and creating fetures.
3. Selecting relevant variables and fetures.
4. Merging the transformed tables into a final dataset.

This process ensures that the data and **features** is properly formatted, aggregated, and cleaned for further analysis or machine learning applications. 

---

### Creating the transcript table as `transcript_b`

Clean, normalizing and transforming column values from the transcript orighinal table:

- Copy the original data.
- Normalizing the `event` column values replacing `' '` --> `'_'`. 
- Normalizing the `value` column dictionarie keys `offer id` --> `offer_id`.
- Creating the `transcript_b` table using `json_normalize` function.
- Creating `ofr_id_short` column with an id more readeble.
- Fill NaN with apropriate values.
- Creating a `tag` column to identify the person-event-offer interactions one by one. (as a person can interact with the same offer more than once)

In [40]:
transcript = transcript_raw.copy(deep=True)

transcript['event'] = transcript['event'].str.replace(' ','_')

def fix_offer_id(value):
    if isinstance(value, dict) and 'offer id' in value:
        value['offer_id'] = value.pop('offer id')
    return value

transcript['value'] = transcript['value'].apply(fix_offer_id)

value_df = pd.json_normalize(transcript['value']) 
transcript_b = pd.concat([transcript, value_df], axis=1).drop('value', axis=1)

port_id = {
    'ae264e3637204a6fb9bb56bc8210ddfd': 'ofr_A',
    '4d5c57ea9a6940dd891ad53e9dbe8da0': 'ofr_B',
    '3f207df678b143eea3cee63160fa8bed': 'ofr_C',
    '9b98b8c7a33c4b65b9aebfe6a799e6d9': 'ofr_D',
    '0b1e1539f2cc45b7b9fa7c272da2e1d7': 'ofr_E',
    '2298d6c36e964ae4a3e7e9706d1fb8c2': 'ofr_F',
    'fafdcd668e3743c1bb461111dcafc2a4': 'ofr_G',
    '5a8bc65990b245e5a138643cd4eb9837': 'ofr_H',
    'f19421c1d4aa40978ebb69ca19b0e20d': 'ofr_I',
    '2906b810c7d4411798c6938adc9daaa5': 'ofr_J'
}

transcript_b['ofr_id_short'] = transcript_b['offer_id'].map(port_id)


transcript_b['ofr_id_short'] = transcript_b['ofr_id_short'].fillna('tran')
transcript_b['amount'] = transcript_b['amount'].fillna(0)
transcript_b['reward'] = transcript_b['reward'].fillna(0)

transcript_b = transcript_b.drop(columns = ['offer_id'])

transcript_b['tag'] = (
    transcript_b.groupby(['person', 'ofr_id_short', 'event'])
    .cumcount()
)

transcript_b.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306534 entries, 0 to 306533
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   person        306534 non-null  object 
 1   event         306534 non-null  object 
 2   time          306534 non-null  int64  
 3   amount        306534 non-null  float64
 4   reward        306534 non-null  float64
 5   ofr_id_short  306534 non-null  object 
 6   tag           306534 non-null  int64  
dtypes: float64(2), int64(2), object(3)
memory usage: 16.4+ MB


### Separating the events and transactions from transcript data

- creating `events` and `transactions` dataframes
- Dropping columns that are not needed

In [41]:
events = transcript_b[transcript_b['event'] != 'transaction'].copy()
events = events.drop(columns=['amount']) # drop the amount column as it contais zero for all rows.

transactions = transcript_b[transcript_b['event'] == 'transaction'].copy()
transactions = transactions.drop(columns=['reward',	'ofr_id_short']) # drop the reward (all zeros) and offer_id columns as they are not relevant. 

In [42]:
events.info()

<class 'pandas.core.frame.DataFrame'>
Index: 167581 entries, 0 to 306527
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   person        167581 non-null  object 
 1   event         167581 non-null  object 
 2   time          167581 non-null  int64  
 3   reward        167581 non-null  float64
 4   ofr_id_short  167581 non-null  object 
 5   tag           167581 non-null  int64  
dtypes: float64(1), int64(2), object(3)
memory usage: 8.9+ MB


In [43]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
Index: 138953 entries, 12654 to 306533
Data columns (total 5 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   person  138953 non-null  object 
 1   event   138953 non-null  object 
 2   time    138953 non-null  int64  
 3   amount  138953 non-null  float64
 4   tag     138953 non-null  int64  
dtypes: float64(1), int64(2), object(2)
memory usage: 6.4+ MB


In [44]:
events

,person,event,time,reward,ofr_id_short,tag
0,78afa995795e4d85b5d9ceeca43f5fef,offer_received,0,0.0,ofr_D,0
1,a03223e636434f42ac4c3df47e8bac43,offer_received,0,0.0,ofr_E,0
2,e2127556f4f64592b11af22de27a7932,offer_received,0,0.0,ofr_J,0
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer_received,0,0.0,ofr_G,0
4,68617ca6246f4fbc85e91a2a49552598,offer_received,0,0.0,ofr_B,0
...,...,...,...,...,...,...
306497,a6f84f4e976f44508c358cc9aba6d2b3,offer_completed,714,3.0,ofr_F,0
306506,b895c57e8cd047a8872ce02aa54759d6,offer_completed,714,2.0,ofr_G,1
306507,8dda575c2a1d44b9ac8e8b07b93d1f8e,offer_viewed,714,0.0,ofr_E,0
306509,8431c16f8e1d440880db371a68f82dd0,offer_completed,714,2.0,ofr_G,0


In [45]:
transactions

,person,event,time,amount,tag
12654,02c083884c7d45b39cc68e1314fec56c,transaction,0,0.83,0
12657,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,transaction,0,34.56,0
12659,54890f68699049c2a04d415abc25e717,transaction,0,13.23,0
12670,b2f1cd155b864803ad8334cdf13c4bd2,transaction,0,19.51,0
12671,fe97aa22dd3e48c8b143116a8403dd52,transaction,0,18.97,0
...,...,...,...,...,...
306529,b3a1272bc9904337b331bf348c3e8c17,transaction,714,1.59,13
306530,68213b08d99a4ae1b0dcb72aebd9aa35,transaction,714,9.53,1
306531,a00058cf10334a308c68e7631c529907,transaction,714,3.61,19
306532,76ddbd6576844afe811f1a3c0fbb5bec,transaction,714,3.53,12


## Creating `event_f` **Feature Table**
Tracking person-offer-event time-based interactions to analyze user engagement and responsiveness.

### Facts Definition:
- Each `person-event-offer-tag` is grouped and stacked into event columns, placing `time` and `reward` values accordingly.
- The 'Tag' column ensures the uniqueness of interactions and prevents aggregation when a person interacts with the same offer type more than once.

### Feature Strategy:

#### Handling Missing Values
- `time_offer_completed`, `time_offer_received`, and `time_offer_viewed` are filled with `np.inf` to signify missing events, avoiding artificial zeroes.

#### Engagement Metrics
Key features quantify user responsiveness:

- **Time Differences:** (help features - not included in the final dataset)
  - `to_vr`: Time from receiving to viewing `{0 to inf}` (delays in viewing).
  - `to_cv`: Time from viewing to completion `{-inf to inf}` (delays in completions after viewing).
  - `to_cr`: Time from receiving to completion `{0 to inf}` (delays in completion, if viewed or not).

- **Inverse Time-Based Scores (`1/(x+1) * 100`):** (calculated using the previous time differences).
  - `curiosity_vr`: `{0 to 100}` Measures _speed_ of viewing after receiving.
  - `eagerness_cv`: `{0 to 100} or -1` Measures _speed_ of completion after viewing (`-1` indicates completion before viewing).
  - `overall_cr`: `{0 to 100}` Overall responsiveness from receiving to completion.

- **Counts:**
  - `count_offer_completed`: Count of completed offers.
  - `count_offer_received`: Count of received offers. 
  - `count_offer_viewed`: Count of viewed offers.

#### Influence Metrics
- **`influence`**: Binary flag (1/0) indicating if an offer was completed after viewing.
- **`ext_influence`**: Binary flag capturing simultaneous viewing and completion (extreme responsiveness).

In [46]:
# Grouping events by 'person', 'ofr_id_short', 'event', and 'tag',
# then aggregating the maximum 'time' and 'reward' per group
# and restructuring the DataFrame.
events_f = (
    events.groupby(['person', 'event', 'ofr_id_short', 'tag'], dropna=False)
    .agg(
        time=('time', 'max'),
        reward=('reward', 'max')
    )
    .unstack(level=1)  # Unstacking by 'event' to widen the DataFrame
    .reset_index()
)

# Flattening the multi-level column names
events_f.columns = ['_'.join(col).strip('_') for col in events_f.columns.to_flat_index()]

# Dropping unnecessary reward columns
events_f = events_f.drop(columns=['reward_offer_received', 'reward_offer_viewed'])

# Replacing NaN values with infinity to facilitate time calculations
events_f['time_offer_completed'] = events_f['time_offer_completed'].fillna(np.inf)
events_f['time_offer_received'] = events_f['time_offer_received'].fillna(np.inf)
events_f['time_offer_viewed'] = events_f['time_offer_viewed'].fillna(np.inf)

# Computing time differences for various stages of the offer lifecycle
events_f['to_vr'] = events_f['time_offer_viewed'] - events_f['time_offer_received']
events_f['to_cv'] = events_f['time_offer_completed'] - events_f['time_offer_viewed']
events_f['to_cr'] = events_f['time_offer_completed'] - events_f['time_offer_received']

# Calculating curiosity, eagerness, and overall responsiveness scores
events_f['curiosity_vr'] = (1 / (events_f['to_vr'] + 1)) * 100
events_f['eagerness_cv'] = [(1 / (x + 1))*100 if ((not np.isnan(x)) & (x >=0)) else -1 for x in events_f['to_cv']]
events_f['overall_cr'] = (1 / (events_f['to_cr'] + 1)) * 100

# Defining influence metrics based on time conditions
events_f['influence'] = (
    (~pd.isna(events_f['time_offer_completed'])) & 
    (events_f['time_offer_completed'] != np.inf) &
    (events_f['time_offer_completed'] >= events_f['time_offer_viewed'])
).astype(int)

# Extreme influence is when the offer is viewed and completed at the same time (up to 1 hour).
events_f['ext_influence'] = (
    (~pd.isna(events_f['time_offer_viewed'])) & 
    (events_f['time_offer_viewed'] != np.inf) & 
    (events_f['time_offer_completed'] == events_f['time_offer_viewed'])
).astype(int)

# Dropping intermediate time calculation columns and rounding final values
events_f = events_f.drop(
    columns=['time_offer_completed', 'time_offer_received', 'time_offer_viewed', 'to_vr', 'to_cv', 'to_cr']
).round(1)

# Merging with the original events DataFrame to get the count of each event grouped per person and offer, stacked by event.
events_counts = events.groupby(['person', 'ofr_id_short', 'event'], dropna=False).agg(
    count=('event', 'count')
).unstack(level=2).reset_index()

events_counts.columns = ['_'.join(col).strip('_') for col in events_counts.columns.to_flat_index()]

events_f = events_f.merge(events_counts, on=['person', 'ofr_id_short'], how='left')


# Filling NaN values in columns with 0
events_f = events_f.fillna(0)

# Displaying the final DataFrame
events_f

,person,ofr_id_short,tag,reward_offer_completed,curiosity_vr,eagerness_cv,overall_cr,influence,ext_influence,count_offer_completed,count_offer_received,count_offer_viewed
0,0009655768c64bdeb2e877511632db8f,ofr_C,0,0.0,2.7,0.0,0.0,0,0,0.0,1.0,1.0
1,0009655768c64bdeb2e877511632db8f,ofr_G,0,2.0,2.7,-1.0,4.0,0,0,1.0,1.0,1.0
2,0009655768c64bdeb2e877511632db8f,ofr_H,0,0.0,4.0,0.0,0.0,0,0,0.0,1.0,1.0
3,0009655768c64bdeb2e877511632db8f,ofr_I,0,5.0,2.0,-1.0,14.3,0,0,1.0,1.0,1.0
4,0009655768c64bdeb2e877511632db8f,ofr_J,0,2.0,0.0,-1.0,100.0,0,0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
76272,ffff82501cea40309d5fdd7edcca4a07,ofr_E,0,5.0,14.3,4.0,3.2,1,0,1.0,1.0,1.0
76273,ffff82501cea40309d5fdd7edcca4a07,ofr_G,0,2.0,14.3,1.8,1.6,1,0,1.0,1.0,1.0
76274,ffff82501cea40309d5fdd7edcca4a07,ofr_J,0,2.0,5.3,3.2,2.0,1,0,3.0,3.0,3.0
76275,ffff82501cea40309d5fdd7edcca4a07,ofr_J,1,2.0,14.3,100.0,14.3,1,1,3.0,3.0,3.0


In [47]:
transac_amount_sumary.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16578 entries, 0009655768c64bdeb2e877511632db8f to ffff82501cea40309d5fdd7edcca4a07
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   tran_count       16578 non-null  int64  
 1   tran_amoun_tot   16578 non-null  float64
 2   tran_amoun_mean  16578 non-null  float64
 3   tran_amoun_max   16578 non-null  float64
 4   tran_amoun_min   16578 non-null  float64
dtypes: float64(4), int64(1)
memory usage: 777.1+ KB


## Event Summary Features

This section of the code aggregates event data at the `person` level, summarizing the frequency and timing of different event types. The goal is to create structured features that describe user interactions with various events over time, which can be useful for behavioral analysis and predictive modeling.

### Feature Aggregation
The code groups data by `person` and `event`, then computes the following summary statistics:

- **Event Count (`cnt`)**: Number of occurrences of each event type per person.
- **Average Event Time (`avg_time`)**: Mean timestamp of each event type per person.
- **Maximum Event Time (`max_time`)**: Latest occurrence timestamp of each event type per person.
- **Minimum Event Time (`min_time`)**: Earliest occurrence timestamp of each event type per person.

### Data Transformation
- The `.unstack(1)` function pivots the `event` column, converting different event types into separate columns.
- The resulting multi-level column names are flattened using list comprehension to create more readable column names.

### Conclusion
These features provide insights into how frequently and when users interact with different events. This structured dataset can be leveraged for customer segmentation, engagement analysis, and recommendation systems.



In [48]:
# counts and time-statistics with focus on offers
offer_person_sumary = transcript_b.groupby(['person', 'ofr_id_short']).agg(
    cnt = ('ofr_id_short', 'count'),
    avg_time = ('time', 'mean'),
    max_time = ('time', 'max'),
    min_time = ('time', 'min')
    
    ).unstack(1).reset_index()  

offer_person_sumary.columns = ['_'.join(col).strip('_') for col in offer_person_sumary.columns.to_flat_index()]

offer_person_sumary.iloc[:,1:12] = offer_person_sumary.iloc[:,1:12].fillna(0)
offer_person_sumary = offer_person_sumary.fillna(1000)


# counts and time-statistics with focus on events
event_person_summary = transcript_b.groupby(['person', 'event']).agg( 
    cnt = ('event', 'count'),
    avg_time = ('time', 'mean'),
    max_time = ('time', 'max'),
    min_time = ('time', 'min')
    
    ).unstack(1).reset_index()  

event_person_summary.columns = ['_'.join(col).strip('_') for col in event_person_summary.columns.to_flat_index()]

event_person_summary.iloc[:,1:5] = event_person_summary.iloc[:,1:5].fillna(0)
event_person_summary = event_person_summary.fillna(1000)

# merging the two dataframes
transcript_features = offer_person_sumary.merge(event_person_summary, on='person', how='left').reset_index(drop=True)

# dropping duplicated columns: ofr_id_short = tran and event = transaction conteins the same information
transcript_features = transcript_features.drop(columns=['min_time_tran', 'avg_time_tran', 'max_time_tran']) 

transcript_features.to_csv('transcript_features.csv')

In [49]:
offer_person_sumary.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17000 entries, 0 to 16999
Data columns (total 45 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   person          17000 non-null  object 
 1   cnt_ofr_A       17000 non-null  float64
 2   cnt_ofr_B       17000 non-null  float64
 3   cnt_ofr_C       17000 non-null  float64
 4   cnt_ofr_D       17000 non-null  float64
 5   cnt_ofr_E       17000 non-null  float64
 6   cnt_ofr_F       17000 non-null  float64
 7   cnt_ofr_G       17000 non-null  float64
 8   cnt_ofr_H       17000 non-null  float64
 9   cnt_ofr_I       17000 non-null  float64
 10  cnt_ofr_J       17000 non-null  float64
 11  cnt_tran        17000 non-null  float64
 12  avg_time_ofr_A  17000 non-null  float64
 13  avg_time_ofr_B  17000 non-null  float64
 14  avg_time_ofr_C  17000 non-null  float64
 15  avg_time_ofr_D  17000 non-null  float64
 16  avg_time_ofr_E  17000 non-null  float64
 17  avg_time_ofr_F  17000 non-null 

In [50]:
event_person_summary.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17000 entries, 0 to 16999
Data columns (total 17 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   person                    17000 non-null  object 
 1   cnt_offer_completed       17000 non-null  float64
 2   cnt_offer_received        17000 non-null  float64
 3   cnt_offer_viewed          17000 non-null  float64
 4   cnt_transaction           17000 non-null  float64
 5   avg_time_offer_completed  17000 non-null  float64
 6   avg_time_offer_received   17000 non-null  float64
 7   avg_time_offer_viewed     17000 non-null  float64
 8   avg_time_transaction      17000 non-null  float64
 9   max_time_offer_completed  17000 non-null  float64
 10  max_time_offer_received   17000 non-null  float64
 11  max_time_offer_viewed     17000 non-null  float64
 12  max_time_transaction      17000 non-null  float64
 13  min_time_offer_completed  17000 non-null  float64
 14  min_ti

In [51]:
transcript_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17000 entries, 0 to 16999
Data columns (total 58 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   person                    17000 non-null  object 
 1   cnt_ofr_A                 17000 non-null  float64
 2   cnt_ofr_B                 17000 non-null  float64
 3   cnt_ofr_C                 17000 non-null  float64
 4   cnt_ofr_D                 17000 non-null  float64
 5   cnt_ofr_E                 17000 non-null  float64
 6   cnt_ofr_F                 17000 non-null  float64
 7   cnt_ofr_G                 17000 non-null  float64
 8   cnt_ofr_H                 17000 non-null  float64
 9   cnt_ofr_I                 17000 non-null  float64
 10  cnt_ofr_J                 17000 non-null  float64
 11  cnt_tran                  17000 non-null  float64
 12  avg_time_ofr_A            17000 non-null  float64
 13  avg_time_ofr_B            17000 non-null  float64
 14  avg_ti

## Transaction Summary Features

This section of the code aggregates transaction data at the `person` level, creating summary statistics that describe each individual's spending behavior. The goal is to generate features that help analyze transaction patterns.

### Feature Aggregation
The code groups transactions by `person` and computes various summary statistics on the `amount` spent:

- **Total Transaction Amount (`tran_amoun_tot`)**: Sum of all transaction amounts for each person.
- **Average Transaction Amount (`tran_amoun_mean`)**: Mean transaction amount per person, indicating typical spending behavior.
- **Maximum Transaction Amount (`tran_amoun_max`)**: Highest transaction amount recorded for each person.
- **Minimum Transaction Amount (`tran_amoun_min`)**: Lowest transaction amount recorded for each person.

### Additional Considerations
- `tran_amount_std` (standard deviation of transaction amounts), are commented out. 
- The `.round(1)` function rounds all numerical values to one decimal place for cleaner representation.

In [52]:
transac_amount_sumary = transactions.groupby(['person'], dropna=False).agg(
    tran_count=('amount', 'count'),
    tran_amoun_tot =('amount', 'sum'),
    tran_amoun_mean=('amount', 'mean'),
    tran_amoun_max=('amount', 'max'),
    tran_amoun_min=('amount', 'min'),
    # tran_amount_std = ('amount', 'std')
).round(1)

In [53]:
person_activity = events_f.merge(transcript_features, on='person', how='left')

person_activity = person_activity.merge(transac_amount_sumary, on='person', how='left').fillna(0)

person_activity_profile = person_activity.merge(profile, left_on='person', right_on='id', how='left').drop(columns=['id'])

person_activity_profile = person_activity_profile.merge(portfolio, on='ofr_id_short', how='left').drop(columns=['id'])


def categorize_age(age):
    if age < 25:
        return 'Young_Adult'
    elif 25 <= age < 40:
        return 'Adult'
    elif 40 <= age < 60:
        return 'Middle_Aged'
    else:
        return 'Senior'
    
person_activity_profile['age_group'] = person_activity_profile['age'].apply(categorize_age)

person_activity_profile.to_csv('person_activity_profile.csv')

person_activity_profile



,person,ofr_id_short,tag,reward_offer_completed,curiosity_vr,eagerness_cv,overall_cr,influence,ext_influence,count_offer_completed,...,age,became_member_on,income,bec_memb_year_month,reward,channels,difficulty,duration,offer_type,age_group
0,0009655768c64bdeb2e877511632db8f,ofr_C,0,0.0,2.7,0.0,0.0,0,0,0.0,...,33,2017-04-21,72000.0,2017-04,0,"[web, email, mobile]",0,4,informational,Adult
1,0009655768c64bdeb2e877511632db8f,ofr_G,0,2.0,2.7,-1.0,4.0,0,0,1.0,...,33,2017-04-21,72000.0,2017-04,2,"[web, email, mobile, social]",10,10,discount,Adult
2,0009655768c64bdeb2e877511632db8f,ofr_H,0,0.0,4.0,0.0,0.0,0,0,0.0,...,33,2017-04-21,72000.0,2017-04,0,"[email, mobile, social]",0,3,informational,Adult
3,0009655768c64bdeb2e877511632db8f,ofr_I,0,5.0,2.0,-1.0,14.3,0,0,1.0,...,33,2017-04-21,72000.0,2017-04,5,"[web, email, mobile, social]",5,5,bogo,Adult
4,0009655768c64bdeb2e877511632db8f,ofr_J,0,2.0,0.0,-1.0,100.0,0,0,1.0,...,33,2017-04-21,72000.0,2017-04,2,"[web, email, mobile]",10,7,discount,Adult
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76272,ffff82501cea40309d5fdd7edcca4a07,ofr_E,0,5.0,14.3,4.0,3.2,1,0,1.0,...,45,2016-11-25,62000.0,2016-11,5,"[web, email]",20,10,discount,Middle_Aged
76273,ffff82501cea40309d5fdd7edcca4a07,ofr_G,0,2.0,14.3,1.8,1.6,1,0,1.0,...,45,2016-11-25,62000.0,2016-11,2,"[web, email, mobile, social]",10,10,discount,Middle_Aged
76274,ffff82501cea40309d5fdd7edcca4a07,ofr_J,0,2.0,5.3,3.2,2.0,1,0,3.0,...,45,2016-11-25,62000.0,2016-11,2,"[web, email, mobile]",10,7,discount,Middle_Aged
76275,ffff82501cea40309d5fdd7edcca4a07,ofr_J,1,2.0,14.3,100.0,14.3,1,1,3.0,...,45,2016-11-25,62000.0,2016-11,2,"[web, email, mobile]",10,7,discount,Middle_Aged


In [54]:
person_activity_profile.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76277 entries, 0 to 76276
Data columns (total 85 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   person                    76277 non-null  object        
 1   ofr_id_short              76277 non-null  object        
 2   tag                       76277 non-null  int64         
 3   reward_offer_completed    76277 non-null  float64       
 4   curiosity_vr              76277 non-null  float64       
 5   eagerness_cv              76277 non-null  float64       
 6   overall_cr                76277 non-null  float64       
 7   influence                 76277 non-null  int64         
 8   ext_influence             76277 non-null  int64         
 9   count_offer_completed     76277 non-null  float64       
 10  count_offer_received      76277 non-null  float64       
 11  count_offer_viewed        76277 non-null  float64       
 12  cnt_ofr_A         